In [12]:
import json
import xml.etree.ElementTree as ET

def convert_xml_to_coco(xml_file):
    # COCO 형식의 JSON 딕셔너리 생성
    CLASSES=['wheelchair',"truck","tree_trunk","traffic_sign",'traffic_light',"traffic_light_controller","table","stroller","stop","scooter","potted_plant","power_controller","pole","person","parking_meter","movable_signage","motorcycle","kiosk","fire_hydrant","dog","chair","cat","carrier","car","bus","bollard","bicycle","bench","barricade"]
    CLASS2IND = {v: i for i, v in enumerate(CLASSES)}
    coco_dict = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    # 카테고리 ID 초기화
    category_id = ''

    # XML 파일 파싱
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 이미지 정보 가져오기
    for image in root.findall('image'):
        image_id = int(image.get('id'))
        image_name = image.get('name')
        image_width = int(image.get('width'))
        image_height = int(image.get('height'))

        # COCO 형식의 이미지 정보 생성
        coco_image = {
            "id": image_id,
            "file_name": image_name,
            "width": image_width,
            "height": image_height
        }

        coco_dict['images'].append(coco_image)

        # 박스 정보 가져오기
        for box in image.findall('box'):
            box_xmin = float(box.get('xtl'))
            box_ymin = float(box.get('ytl'))
            box_xmax = float(box.get('xbr'))
            box_ymax = float(box.get('ybr'))
            box_label = box.get('label')

            category_id = CLASS2IND[box_label]
            # COCO 형식의 어노테이션 정보 생성
            coco_annotation = {
                "id": len(coco_dict['annotations']) + 1,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [box_xmin, box_ymin, int(box_xmax - box_xmin), int(box_ymax - box_ymin)],
                "area": int((box_xmax - box_xmin) * (box_ymax - box_ymin)),
                "iscrowd": 0
            }

            coco_dict['annotations'].append(coco_annotation)

        # 카테고리 정보 생성
        if box_label not in [category['name'] for category in coco_dict['categories']]:
            category = {
                "id": category_id,
                "name": box_label,
                "supercategory": "object"
            }
            coco_dict['categories'].append(category)
            category_id += 1

    return coco_dict

In [19]:
import os
data_root="C:/Users/User/Desktop/"
root=os.listdir(data_root)
root=[f for f in root if f.startswith("Bbox")]
for i in range(len(root)):
    root_folder=os.path.join(data_root,root[i])
    data_folder=os.listdir(root_folder)
    print(data_folder)
    for j in range(len(data_folder)):
        data_dir=os.path.join(root_folder,data_folder[j])
        data_dir=data_dir.replace("\\","/")

        xml_path=os.listdir(data_dir)
        xml_path=[x for x in xml_path if x.endswith("xml")]
        xml_path=os.path.join(data_dir,xml_path[0])
        xml_path=xml_path.replace("\\","/")

        # COCO JSON 파일 생성
        coco_dict = convert_xml_to_coco(xml_path)
        print(data_dir)
        # new_file_name=xml_path.replace(".xml",".json")
        new_file_name=os.path.join(data_dir,"label.json")
        new_file_name=new_file_name.replace("\\","/")
        
        with open(new_file_name, 'w') as f:
            json.dump(coco_dict, f)

['Bbox_1901', 'Bbox_1902', 'Bbox_1905']
C:/Users/User/Desktop/Bbox_23_new/Bbox_1901
C:/Users/User/Desktop/Bbox_23_new/Bbox_1902
C:/Users/User/Desktop/Bbox_23_new/Bbox_1905


In [ ]:
# # XML 파일 경로
# xml_file = "C:/Users/User/Desktop/Bbox_1901/P1014_07.xml"

# # COCO JSON 파일 생성
# coco_dict = convert_xml_to_coco(xml_file)
# with open('C:/Users/User/Desktop/Bbox_1901/label.json', 'w') as f:
#     json.dump(coco_dict, f)